In [77]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'logist8', # имя пользователя
             'pwd': 'cargo', # пароль
             'host': 'localhost',
             'port': 5432, # порт подключения
             'db': 'aircrafts8'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                  db_config['pwd'],
                                                  db_config['host'],
                                                  db_config['port'],
                                                  db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string) 
                       #connect_args={'sslmode':'require'}) 

Запросы

В каком аэропорту запарковано больше всего самолётов

In [73]:
query = """ SELECT airport_id AS "Airport", count(pa.parking_id) as "Aircrafts parked"
            FROM status st 
            LEFT JOIN parking pa on st.parking_id = pa.parking_id
            GROUP BY st.airport_id
            ORDER BY count(pa.parking_id) DESC
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,Airport,Aircrafts parked
0,BOS,9
1,FRA,8
2,DXB,7
3,LED,5
4,LGW,1


У какой авиакомпании больше всего Боингов

In [74]:
query = """ SELECT al.airline_name AS "Airline name", count(acs.aircraft_id) AS "Boing qty"
            FROM airline al 
            LEFT JOIN status st ON al.airline_id = st.airline_id 
            LEFT JOIN aircraft ac ON ac.tail_number = st.tail_number 
            LEFT JOIN aircraftspecs acs on acs.aircraft_id = ac.aircraft_id
            WHERE acs.manufacturer_id = 'BOEING'
            GROUP BY al.airline_name
            ORDER BY count(acs.aircraft_id) DESC
        """

pd.io.sql.read_sql(query, con = engine) 

,Airline name,Boing qty
0,Emirates,6
1,WizzAir,4
2,Lufthansa,2
3,Volga-Dnepr,2
4,American Airline,2


Бортовые номера самолётов 1, 2, 3 партии

In [75]:
query = """ SELECT ac.tail_number AS "Tail number", dt.batch AS "Batch"
            FROM aircraft ac 
            LEFT JOIN dateofissue dt ON dt.date_id = ac.date_id
            WHERE dt.batch = '3' OR dt.batch = '2' OR dt.batch = '1'
            ORDER BY batch 
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,Tail number,Batch
0,N315RJ,1
1,D-AIBH,1
2,A6-EOR,1
3,A6-EPX,1
4,A6-EDP,2
5,RA-82074,2
6,N8001N,2
7,N4005X,2
8,N156UW,2
9,N314RH,2


С каким типом двигателя больше всего самолётов НЕ готово?

In [64]:
query = """ SELECT acs.engine_type AS "Engine type", count(st.readiness) as "Out of order"
            FROM aircraftspecs acs 
            LEFT JOIN aircraft ac on ac.aircraft_id = acs.aircraft_id 
            LEFT JOIN status st on st.tail_number = ac.tail_number 
            WHERE st.readiness = 'maintenance' OR st.readiness = 'repair'
            GROUP BY acs.engine_type
            ORDER BY count(st.readiness = 'maintenance' OR st.readiness = 'repair') DESC
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,Engine type,Out of order
0,P&W,4
1,GE,3
2,Progress,2


ТОП 3 самых дорогих авиакомпаний по стоимости флота и средняя стоимость одного самолёта.

In [76]:
query = """ SELECT al.airline_name AS "Airline name", count(st.tail_number) as "Aircrafts qty", sum(acs.price) as "Fleet price", avg(acs.price) as "Average price"
            FROM airline al 
            LEFT JOIN status st on st.airline_id = al.airline_id 
            LEFT JOIN aircraft ac on ac.tail_number = st.tail_number 
            LEFT JOIN aircraftspecs acs on acs.aircraft_id = ac.aircraft_id
            GROUP BY al.airline_name
            ORDER BY sum(acs.price) DESC
            LIMIT 3
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,Airline name,Aircrafts qty,Fleet price,Average price
0,American Airline,10,1340,134.00
1,Emirates,8,1150,143.75
2,Volga-Dnepr,4,575,143.75


Самолёты какого типа были в аэоропрту Дубая в январе 2022.

In [86]:
query = """ SELECT ac.aircraft_id, st.date_of_arrival as "Date of arrival"
            FROM aircraft ac 
            LEFT JOIN status st on st.tail_number = ac.tail_number
            LEFT JOIN airport ap on ap.airport_id = st.airport_id 
            WHERE st.date_of_arrival >= '2022-01-01' AND st.date_of_arrival < '2022-02-01'
            GROUP BY ac.aircraft_id, st.date_of_arrival
            ORDER BY st.date_of_arrival DESC
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,aircraft_id,Date of arrival
0,B74CG,2022-01-22
1,A32CP,2022-01-19
2,B74CG,2022-01-18
3,A32CP,2022-01-06


Распределение за последние 3 месяца с каким типом двигателя в каждом аэоропорте

In [103]:
query = """ SELECT ap.airport_id as "Airport", acs.engine_type, st.date_of_arrival as "Date"
            FROM airport ap 
            LEFT JOIN status st on st.airport_id = ap.airport_id
            LEFT JOIN aircraft ac on ac.tail_number = st.tail_number
            LEFT JOIN aircraftspecs acs on acs.aircraft_id = ac.aircraft_id
            WHERE st.date_of_arrival >= '2022-09-19'
            GROUP BY ap.airport_id, acs.engine_type, st.date_of_arrival
            ORDER BY ap.airport_id
            ;
        """

pd.io.sql.read_sql(query, con = engine) 

,Airport,engine_type,Date
0,BOS,P&W,2022-10-15
1,BOS,Progress,2022-09-28
2,DXB,GE,2022-11-17
3,FRA,P&W,2022-09-19
4,FRA,P&W,2022-11-05
5,LED,CFM,2022-11-02
6,LED,GE,2022-12-12
